# Βήμα 4: Μηχανή Αναζήτησης
Σε αυτό το βήμα, θα φτιάξουμε τη μηχανή αναζήτησης που θα ψάχνει στα άρθρα μας. Θα υποστηρίξουμε τρεις τρόπους αναζήτησης:

Boolean Αναζήτηση: Επιστρέφει άρθρα που περιέχουν ή δεν περιέχουν συγκεκριμένες λέξεις, ανάλογα με το ερώτημα.
TF-IDF Αναζήτηση: Βαθμολογεί τα άρθρα με βάση τη σημασία των λέξεων και το πόσο ταιριάζουν στο ερώτημα.
BM25 Αναζήτηση: Ένας πιο έξυπνος τρόπος να υπολογίσουμε πόσο καλά ταιριάζει ένα άρθρο στο ερώτημα.
Ο χρήστης μπορεί να επιλέξει ποιον από τους τρεις τρόπους θέλει να χρησιμοποιήσει.

## Κελί 1: Εισαγωγή βιβλιοθηκών
Σχόλιο:
Εισάγουμε τις βιβλιοθήκες που απαιτούνται για την αναζήτηση:
TfidfVectorizer: Για τον υπολογισμό του TF-IDF.
cosine_similarity: Για την κατάταξη με βάση τη συνάφεια.
BM25Okapi: Για την υλοποίηση του BM25.

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rank_bm25 import BM25Okapi
import json

## Κελί 2: Συνάρτηση Boolean Αναζήτησης
Η Boolean Αναζήτηση βρίσκει άρθρα που περιέχουν όλους τους όρους του ερωτήματος.

In [52]:
def boolean_search(query, inverted_index, articles):
    """
    Εκτελεί Boolean αναζήτηση με υποστήριξη AND, OR, και NOT.
    """
    query = query.lower()
    query_tokens = query.split(" ")  # Διαχωρισμός σε λέξεις
    results = set(range(len(articles)))  # Αρχικό σύνολο αποτελεσμάτων (όλα τα άρθρα)

    current_set = set()  # Τρέχον σύνολο αποτελεσμάτων
    operator = "AND"  # Προεπιλεγμένος τελεστής

    for token in query_tokens:
        if token in ["and", "or", "not"]:
            operator = token.upper()  # Ενημέρωση τελεστή
        else:
            # Εύρεση άρθρων που περιέχουν τον τρέχον όρο
            token_results = set(inverted_index.get(token, []))

            # Εφαρμογή του τελεστή
            if operator == "AND":
                current_set &= token_results if current_set else token_results
            elif operator == "OR":
                current_set |= token_results
            elif operator == "NOT":
                current_set -= token_results

    return [articles[idx] for idx in current_set]


## Κελί 3: Συνάρτηση TF-IDF Αναζήτησης
Η TF-IDF Αναζήτηση κατατάσσει τα άρθρα με βάση τη συνάφεια του ερωτήματος.

In [53]:
def tfidf_search(query, articles):
    """
    Εκτελεί TF-IDF αναζήτηση.
    
    Args:
        query (str): Το ερώτημα αναζήτησης.
        articles (list): Η λίστα των άρθρων.
    
    Returns:
        list: Ταξινομημένα άρθρα με βάση το TF-IDF score.
    """
    corpus = [article['content'] for article in articles]
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(corpus)
    query_vector = vectorizer.transform([query])  # Μετατροπή του ερωτήματος σε διάνυσμα
    scores = cosine_similarity(query_vector, tfidf_matrix).flatten()
    ranked_indices = scores.argsort()[::-1]  # Ταξινόμηση κατά φθίνουσα σειρά
    return [articles[idx] for idx in ranked_indices if scores[idx] > 0]


## Κελί 4: Συνάρτηση BM25 Αναζήτησης**
Σχόλιο:
Η BM25 Αναζήτηση κατατάσσει τα άρθρα χρησιμοποιώντας το μοντέλο BM25.

In [54]:
def bm25_search(query, articles):
    """
    Εκτελεί BM25 αναζήτηση.
    
    Args:
        query (str): Το ερώτημα αναζήτησης.
        articles (list): Η λίστα των άρθρων.
    
    Returns:
        list: Ταξινομημένα άρθρα με βάση το BM25 score.
    """
    corpus = [article['content'].split() for article in articles]
    bm25 = BM25Okapi(corpus)
    query_tokens = query.lower().split()  # Διαχωρισμός του ερωτήματος σε λέξεις
    scores = bm25.get_scores(query_tokens)
    ranked_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)
    return [articles[idx] for idx in ranked_indices if scores[idx] > 0]


## Κελί 5: Φόρτωση δεδομένων**
Σχόλιο:
Φορτώνουμε τα δεδομένα από τα αρχεία processed_articles.json και inverted_index.json.

In [33]:
# Φόρτωση των άρθρων και του ανεστραμμένου ευρετηρίου
with open('processed_articles.json', 'r', encoding='utf-8') as f:
    articles = json.load(f)
with open('inverted_index.json', 'r', encoding='utf-8') as f:
    inverted_index = json.load(f)

print(f"Loaded {len(articles)} articles and inverted index.")


Loaded 100 articles and inverted index.


## Κελί 6: Επιλογή αλγορίθμου και εκτέλεση αναζήτησης**
Σχόλιο:
Ο χρήστης εισάγει το ερώτημα και επιλέγει έναν από τους τρεις αλγορίθμους. Εμφανίζονται τα αποτελέσματα βάσει της επιλογής.

In [49]:
# Ερώτημα και επιλογή αλγορίθμου
query = input("Enter your search query: ")
algorithm = input("Choose retrieval algorithm (boolean/tfidf/bm25): ").lower()

# Εκτέλεση αναζήτησης
if algorithm == "boolean":
    results = boolean_search(query, inverted_index, articles)
elif algorithm == "tfidf":
    results = tfidf_search(query, articles)
elif algorithm == "bm25":
    results = bm25_search(query, articles)
else:
    print("Invalid algorithm. Please choose between 'boolean', 'tfidf', or 'bm25'.")
    results = []

# Εμφάνιση αποτελεσμάτων
if results:
    print("\nSearch Results:")
    for result in results:
        print(f"- {result['title']}")
else:
    print("No results found.")

Enter your search query:  algebra
Choose retrieval algorithm (boolean/tfidf/bm25):  bm25



Search Results:
- Algebra
- Homological algebra
- Diophantus
- Mathematics
- History of mathematics
- Combinatorics
- Category theory
- Independence (probability theory)
- Algebraic geometry
- Discrete mathematics
- Foundations of mathematics
- Mathematics education
- Integer
- Mathematical physics
- Applied mathematics
- Mathematical logic
- Scheme (mathematics)
- Lists of mathematics topics
- Wiles's proof of Fermat's Last Theorem
- Mathematical analysis
- Algebraic number theory
- Numerical analysis
- Pure mathematics
- Transcendental number theory
- Computational mathematics
- Greek mathematics
- Abstraction (mathematics)
- Axiom
- Integral
- Set theory
- Euclid
- Variable (mathematics)
- Calculus
- Geomathematics
- Formula
- Leonhard Euler
- Prime number
- Babylonian mathematics
- Number
- Axiomatic system
- Mathematical economics
- Geometry
- Number theory
- Arithmetic
- Carl Friedrich Gauss
- Natural number
- Fermat's Last Theorem
- Differential calculus
- Probability
- Diophan

# Boolean OR
Enter your search query:  algebra or calculus
Choose retrieval algorithm (boolean/tfidf/bm25):  boolean

Search Results:
- Mathematics
- History of mathematics
- Lists of mathematics topics
- Glossary of areas of mathematics
- Geometry
- Algebra
- Calculus
- Mathematical analysis
- Discrete mathematics
- Mathematical logic
- Set theory
- Mathematical physics
- Mathematical and theoretical biology
- Mathematical economics
- Philosophy of mathematics
- Mathematics education
- Theorem
- Science
- Axiom
- Rule of inference
- Computer science
- Pure mathematics
- Rigour
- Arithmetic
- Foundations of mathematics
- Formula
- Differential calculus
- Integral
- Continuous function
- Variable (mathematics)
- Number
- Pierre de Fermat
- Leonhard Euler
- Carl Friedrich Gauss
- Algebraic geometry
- Category theory

# TFIDF
Enter your search query:  algebra
Choose retrieval algorithm (boolean/tfidf/bm25):  tfidf

Search Results:
- Algebra
- Homological algebra
- Diophantus
- Mathematics
- History of mathematics
- Computational mathematics
- Combinatorics
- Foundations of mathematics
- Category theory
- Lists of mathematics topics
- Mathematics education
- Mathematical physics
- Abstraction (mathematics)
- Discrete mathematics
- Algebraic geometry
- Mathematical analysis
- Pure mathematics
- Applied mathematics
- Integer
- Mathematical logic
- Greek mathematics
- Numerical analysis
- Independence (probability theory)
- Algebraic number theory
- Wiles's proof of Fermat's Last Theorem
- Transcendental number theory
- Scheme (mathematics)
- Euclid
- Axiom
- Geomathematics
- Set theory
- Variable (mathematics)
- Calculus
- Integral
- Formula
- Babylonian mathematics
- Axiomatic system
- Number theory
- Leonhard Euler
- Number
- Geometry
- Mathematical economics
- Arithmetic
- Prime number
- Carl Friedrich Gauss
- Natural number
- Fermat's Last Theorem
- Probability
- Diophantine equation
- Differential calculus
- Natural science
- Philosophy of mathematics
- Statistics
- Fraction

# BM25
Enter your search query:  algebra
Choose retrieval algorithm (boolean/tfidf/bm25):  bm25

Search Results:
- Algebra
- Homological algebra
- Diophantus
- Mathematics
- History of mathematics
- Combinatorics
- Category theory
- Independence (probability theory)
- Algebraic geometry
- Discrete mathematics
- Foundations of mathematics
- Mathematics education
- Integer
- Mathematical physics
- Applied mathematics
- Mathematical logic
- Scheme (mathematics)
- Lists of mathematics topics
- Wiles's proof of Fermat's Last Theorem
- Mathematical analysis
- Algebraic number theory
- Numerical analysis
- Pure mathematics
- Transcendental number theory
- Computational mathematics
- Greek mathematics
- Abstraction (mathematics)
- Axiom
- Integral
- Set theory
- Euclid
- Variable (mathematics)
- Calculus
- Geomathematics
- Formula
- Leonhard Euler
- Prime number
- Babylonian mathematics
- Number
- Axiomatic system
- Mathematical economics
- Geometry
- Number theory
- Arithmetic
- Carl Friedrich Gauss
- Natural number
- Fermat's Last Theorem
- Differential calculus
- Probability
- Diophantine equation
- Natural science
- Statistics
- Fraction
- Philosophy of mathematics

# Αποθήκευση αποτελεσμάτων αναζήτησης

In [41]:
# Αποθήκευση αποτελεσμάτων αναζήτησης
search_results = {
    "query": query,
    "algorithm": algorithm,
    "retrieved_ids": [articles.index(doc) for doc in results]
}

with open('search_results.json', 'w', encoding='utf-8') as f:
    json.dump(search_results, f, indent=4, ensure_ascii=False)

print("Search results saved to 'search_results.json'.")


Search results saved to 'search_results.json'.
